In [85]:
import pandas as pd

In [86]:
# acs_pums_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/csv_pmn_one_year/psam_p27.csv')
acs_pums_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/raw_data/Minnesota - ACS PUMS - Five-Years/psam_p27.csv')
# /content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/raw_data/Minnesota - ACS PUMS - Five-Years/psam_p27.csv
df_lookup = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/reference_data/Pums Ref Data - Sheet1.csv')

pumas_to_cities_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/Geocorr from the Missouri Census Data Center - Minnesota - pumas_to_cities.csv')
pumas_to_metros_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/Geocorr from the Missouri Census Data Center - Minnesota - pumas_to_metros.csv')

# acs_pums_df.head()

In [87]:
acs_pums_df.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA10,PUMA20,REGION,ST,ADJINC,PWGTP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2018GQ0000054,4,1,1900,-9,2,27,1184371,4,...,1,4,4,7,4,4,4,4,7,3
1,P,2018GQ0000077,4,1,2300,-9,2,27,1184371,16,...,15,19,31,17,4,4,4,17,15,27
2,P,2018GQ0000094,4,1,1410,-9,2,27,1184371,3,...,3,1,0,0,0,2,3,1,2,1
3,P,2018GQ0000141,4,1,1304,-9,2,27,1184371,3,...,0,6,4,4,5,3,3,3,4,0
4,P,2018GQ0000370,4,1,900,-9,2,27,1184371,7,...,8,0,0,0,8,6,7,7,8,1


In [88]:
raw_st_paul_filtered_df = acs_pums_df[acs_pums_df['PUMA20'].isin([1503, 1504, 1505])]

In [89]:
raw_st_paul_filtered_df.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA10,PUMA20,REGION,ST,ADJINC,PWGTP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
225213,P,2022GQ0000897,4,1,-9,1503,2,27,1042311,13,...,15,15,13,11,14,13,14,12,17,13
225226,P,2022GQ0001420,4,1,-9,1503,2,27,1042311,1,...,1,2,1,1,2,2,2,1,1,2
225228,P,2022GQ0001538,4,1,-9,1505,2,27,1042311,1,...,2,8,1,1,0,1,1,0,0,0
225231,P,2022GQ0001761,4,1,-9,1504,2,27,1042311,32,...,33,35,33,33,33,33,33,33,31,33
225251,P,2022GQ0002859,4,1,-9,1504,2,27,1042311,5,...,1,10,7,0,7,5,1,5,1,8


In [90]:
acs_pums_columns_to_keep = [
    'RT', 'SERIALNO', 'DIVISION', 'SPORDER', 'PUMA20', 'REGION', 'ST', 'AGEP',
    'SOCP', 'RAC1P', 'SEX', 'SCHL', 'PINCP', 'ADJINC', 'WAGP', 'PWGTP',
]


acs_pums_df = acs_pums_df.astype(str)
acs_pums_df['SCHL'] = acs_pums_df['SCHL'].str.replace('.0', '')

acs_pums_df = acs_pums_df[acs_pums_columns_to_keep]
acs_pums_df.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA20,REGION,ST,AGEP,SOCP,RAC1P,SEX,SCHL,PINCP,ADJINC,WAGP,PWGTP
0,P,2018GQ0000054,4,1,-9,2,27,36,472140,1,1,17,2000.0,1184371,2000.0,4
1,P,2018GQ0000077,4,1,-9,2,27,56,nan,5,1,17,0.0,1184371,0.0,16
2,P,2018GQ0000094,4,1,-9,2,27,21,399032,1,2,19,1300.0,1184371,1300.0,3
3,P,2018GQ0000141,4,1,-9,2,27,20,nan,1,2,15,4720.0,1184371,0.0,3
4,P,2018GQ0000370,4,1,-9,2,27,46,2310XX,1,2,24,130000.0,1184371,130000.0,7


In [91]:
lookup_dict = df_lookup.groupby('VARIABLE_NAME').apply(lambda x: x.set_index('CODE')['VALUE'].to_dict()).to_dict()

# Function to replace codes with values based on the mapping
def replace_codes(df, column, code_to_value):
    if column in code_to_value:
        df[column] = df[column].map(code_to_value[column]).fillna(df[column])
    return df

# lookup_dict

In [92]:
# Apply the replacement for each column in df_psam_p27
for column in acs_pums_df.columns:
    acs_pums_df = replace_codes(acs_pums_df, column, lookup_dict)

<ipython-input-91-51672bf166fe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].map(code_to_value[column]).fillna(df[column])
<ipython-input-91-51672bf166fe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].map(code_to_value[column]).fillna(df[column])
<ipython-input-91-51672bf166fe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [93]:
len(acs_pums_df)

284189

In [94]:
# geos_columns_to_keep = ['PUMA (2022)', 'Place name', 'PUMA22 name_x', '2020 CBSA name']
# geos_merged_df = pumas_to_cities_df.merge(pumas_to_metros_df, on='PUMA (2022)', how='inner')
# geos_merged_df = geos_merged_df[geos_columns_to_keep]
# geos_merged_df['PUMA (2022)'] = geos_merged_df['PUMA (2022)'].astype(str)
# geos_merged_df.head()

In [95]:
# len(geos_merged_df)

In [96]:
# acs_pums_geos_merged_df = pd.merge(acs_pums_df, geos_merged_df, left_on='PUMA', right_on='PUMA (2022)', how='left')
# acs_pums_geos_merged_df.head()

In [97]:
# len(acs_pums_geos_merged_df)

In [98]:
green_jobs_df = pd.read_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/processed_data/Green Enhanced Skills Occupations - All - Sheet1.csv')

green_jobs_df.head()

,Reported Occupation,O*NET-SOC Code,O*NET-SOC Title,O*NET-SOC Category,O*NET-SOC Sector
0,Coal Gasification Technicians**,51-8013.00,Power Plant Operators,Green Enhanced Skills,Renewable Energy Generation
1,Coal Miners**,47-5041.00,Continuous Mining Machine Operators,Green Enhanced Skills,Renewable Energy Generation
2,Cogeneration Technicians,51-8013.00,Power Plant Operators,Green Enhanced Skills,Renewable Energy Generation
3,Combustion Technicians,51-8013.00,Power Plant Operators,Green Enhanced Skills,Renewable Energy Generation
4,Electrical Energy Storage/Distribution Technic...,11-3071.02,Storage and Distribution Managers,Green Enhanced Skills,Renewable Energy Generation


In [99]:
green_jobs_df_selected = green_jobs_df[['O*NET-SOC Title', 'O*NET-SOC Code']]
green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.split('.').str[0]
green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.replace('-', '')
# Get unique rows based on the selected columns
green_jobs_df_unique = green_jobs_df_selected.drop_duplicates()


# Display the first few rows of the unique DataFrame
print(green_jobs_df_unique.head(10))

                            O*NET-SOC Title O*NET-SOC Code
0                     Power Plant Operators         518013
1       Continuous Mining Machine Operators         475041
4         Storage and Distribution Managers         113071
5   Maintenance and Repair Workers, General         499042
7                      Electrical Engineers         172071
8                                Machinists         514041
10                     Mechanical Engineers         172141
11                      Sheet Metal Workers         472211
21  Nuclear Equipment Operation Technicians         194051
22          Nuclear Power Reactor Operators         518011


<ipython-input-99-bf50efa47340>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.split('.').str[0]
<ipython-input-99-bf50efa47340>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_jobs_df_selected['O*NET-SOC Code'] = green_jobs_df_selected['O*NET-SOC Code'].str.replace('-', '')


In [100]:
green_jobs_df_unique = green_jobs_df_unique.groupby('O*NET-SOC Code').agg({'O*NET-SOC Title': 'first'}).reset_index()

In [101]:
green_jobs_acs_df_merged = pd.merge(acs_pums_df, green_jobs_df_unique, left_on='SOCP', right_on='O*NET-SOC Code', how='left')

# Display the first few rows of the merged DataFrame
print(green_jobs_acs_df_merged.head())

  RT       SERIALNO DIVISION SPORDER PUMA20 REGION  ST AGEP    SOCP  \
0  P  2018GQ0000054        4       1     -9      2  27   36  472140   
1  P  2018GQ0000077        4       1     -9      2  27   56     nan   
2  P  2018GQ0000094        4       1     -9      2  27   21  399032   
3  P  2018GQ0000141        4       1     -9      2  27   20     nan   
4  P  2018GQ0000370        4       1     -9      2  27   46  2310XX   

                                               RAC1P     SEX  \
0                                        White alone    Male   
1  American Indian and Alaska Native tribes speci...    Male   
2                                        White alone  Female   
3                                        White alone  Female   
4                                        White alone  Female   

                                           SCHL     PINCP   ADJINC      WAGP  \
0                 GED or alternative credential    2000.0  1184371    2000.0   
1                 GED or alt

In [102]:
green_jobs_acs_df_merged_filtered = green_jobs_acs_df_merged.dropna(subset=['O*NET-SOC Title', 'O*NET-SOC Code'])
len(green_jobs_acs_df_merged_filtered)
len(green_jobs_acs_df_merged)

284189

In [103]:
green_jobs_acs_df_merged['Green Job Flag'] = green_jobs_acs_df_merged['O*NET-SOC Code'].notnull().astype(int)

In [104]:
flag_counts = green_jobs_acs_df_merged['Green Job Flag'].value_counts()

# Display the counts
print(flag_counts)

Green Job Flag
0    269519
1     14670
Name: count, dtype: int64


In [105]:
green_jobs_acs_df_merged.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA20,REGION,ST,AGEP,SOCP,RAC1P,SEX,SCHL,PINCP,ADJINC,WAGP,PWGTP,O*NET-SOC Code,O*NET-SOC Title,Green Job Flag
0,P,2018GQ0000054,4,1,-9,2,27,36,472140,White alone,Male,GED or alternative credential,2000.0,1184371,2000.0,4,NaN,NaN,0
1,P,2018GQ0000077,4,1,-9,2,27,56,nan,American Indian and Alaska Native tribes speci...,Male,GED or alternative credential,0.0,1184371,0.0,16,NaN,NaN,0
2,P,2018GQ0000094,4,1,-9,2,27,21,399032,White alone,Female,"1 or more years of college credit, no degree",1300.0,1184371,1300.0,3,NaN,NaN,0
3,P,2018GQ0000141,4,1,-9,2,27,20,nan,White alone,Female,12th grade - no diploma,4720.0,1184371,0.0,3,NaN,NaN,0
4,P,2018GQ0000370,4,1,-9,2,27,46,2310XX,White alone,Female,Doctorate degree,130000.0,1184371,130000.0,7,NaN,NaN,0


In [106]:
st_paul_filtered_df = green_jobs_acs_df_merged[green_jobs_acs_df_merged['PUMA20'].isin(['1503', '1504', '1505'])]

# Display the first few rows of the filtered DataFrame
print(st_paul_filtered_df.head())

       RT       SERIALNO DIVISION SPORDER PUMA20 REGION  ST AGEP    SOCP  \
225213  P  2022GQ0000897        4       1   1503      2  27   16     nan   
225226  P  2022GQ0001420        4       1   1503      2  27   31     nan   
225228  P  2022GQ0001538        4       1   1505      2  27   54  472181   
225231  P  2022GQ0001761        4       1   1504      2  27   80     nan   
225251  P  2022GQ0002859        4       1   1504      2  27   20  434121   

                                  RAC1P     SEX  \
225213  Black or African American alone  Female   
225226                      White alone  Female   
225228                      White alone    Male   
225231                      White alone  Female   
225251                Two or More Races  Female   

                                                SCHL    PINCP   ADJINC  \
225213                                       Grade 9      0.0  1042311   
225226                 GED or alternative credential    100.0  1042311   
225228        

In [107]:
flag_counts = st_paul_filtered_df['Green Job Flag'].value_counts()

# Display the counts
print(flag_counts)

Green Job Flag
0    1658
1      72
Name: count, dtype: int64


In [108]:
st_paul_filtered_df.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA20,REGION,ST,AGEP,SOCP,RAC1P,SEX,SCHL,PINCP,ADJINC,WAGP,PWGTP,O*NET-SOC Code,O*NET-SOC Title,Green Job Flag
225213,P,2022GQ0000897,4,1,1503,2,27,16,nan,Black or African American alone,Female,Grade 9,0.0,1042311,0.0,13,NaN,NaN,0
225226,P,2022GQ0001420,4,1,1503,2,27,31,nan,White alone,Female,GED or alternative credential,100.0,1042311,0.0,1,NaN,NaN,0
225228,P,2022GQ0001538,4,1,1505,2,27,54,472181,White alone,Male,GED or alternative credential,0.0,1042311,0.0,1,472181,Roofers,1
225231,P,2022GQ0001761,4,1,1504,2,27,80,nan,White alone,Female,Master's degree,18000.0,1042311,0.0,32,NaN,NaN,0
225251,P,2022GQ0002859,4,1,1504,2,27,20,434121,Two or More Races,Female,"1 or more years of college credit, no degree",14000.0,1042311,6000.0,5,NaN,NaN,0


In [109]:
st_paul_filtered_df.to_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/processed_data/St_Paul_ACS_All_Jobs.csv', index=False)

In [110]:
# raw_st_paul_filtered_df.to_csv('/content/drive/MyDrive/Upwork - St. Paul Clean Energy Jobs/st_paul_green_jobs_data/raw_data/st_paul_level/ACS-PUMS-StPaul.csv', index=False)